### 1)
a) Determine model speed ignoring frictional resistance

In [1]:
l_m = 1.75 # m

l_p = 280  # m
l_s = l_m / l_p
u_p = 9.003 # m/s
u_m = u_p * (l_s)**(1/2)
print("model speed: ", u_m, " m/s")

model speed:  0.711749644362398  m/s


b) Find total resistance
- Use Reynolds number to model frictional drag for prototype and model
    1. Calculate Reynolds number
    2. Calculate $C_f(Re)$
    3. Find wetted surface area of model
    

In [2]:
import numpy as np
import CoolProp.CoolProp as CP # fluidic properties library

def cf(re):
    """
    Find coefficient of friction due to wetted surface area
    Equation taken from I.T.T.C. referenced in Kundu and Cohen 4th
        re - Reynolds number
    Returns: coefficient of drag
    """
    cf = 0.075 / (np.log10(re)-2)**2
    return cf


def rf(re, rho, u, a):
    """
    Find resistance due to friction
        re - Reynolds number
        rho - density [kg/m^3]
        u - velocity
        a - wetted surface area
    Returns: frictional resistance force
    """
    cf_ = cf(re)
    rf = cf_ * 1/2 * rho * u**2 * a
    return rf


Estimate resistance due to friction  
1) Find viscosity `mu`  
2) Find Reynolds number `re`

In [3]:
# Define atmospheric pressure
p = 101325 # atmospheric pressure  (Pa)

# Model
mu_m = CP.PropsSI('V', 'P', p, 'T', 26+273.25, 'Water')
rho_m = CP.PropsSI('D', 'P', p, 'T', 26+273.25, 'Water')
nu_m = mu_m / rho_m
re_m = (u_m * l_m) / nu_m
print(re_m)

# Prototype
mu_p = CP.PropsSI('V', 'P', p, 'T', 14+273.25, 'INCOMP::MITSW[0.033]')
rho_p = CP.PropsSI('D', 'P', p, 'T', 14+273.25, 'INCOMP::MITSW[0.033]')
nu_p = mu_p/rho_p
re_p = (u_p * l_p) / nu_p
print(re_p)



1430069.6919531708
2058698800.1223087


Find model area assuming flat plate. Calculate resistance.

In [4]:
a_p = 18000 # m^2
a_m = a_p * (l_m / l_p)**2# m^2

rf_m = rf(re_m, rho_m, u_m, a_m)
rf_p = rf(re_p, rho_p, u_p, a_p)

In [5]:
rf_p

1048040.1109668601

Find residual resistance based on measured

In [6]:
# Measured resistance
rm_m = 0.95 # [N]
rr_m = rm_m - rf_m

rr_p = rr_m * (rho_p/rho_m) * (l_p/l_m)**2 * (u_p/u_m)**2

In [7]:
r_p = rr_p + rf_p # N
r_p

1801432.5183163173

$$P = R_p \times U_p$$

In [8]:
r_p * u_p # N m/s

16218296.962401805

Total power required is 16.2 MW